## Author: Farhan Ahmad <farhanhubble@gmail.com>
## 🐦 : [@farhanhubble](https://twitter.com/farhanhubble)
## LinkedIN: https://www.linkedin.com/in/farhanhubble/
--- 

# Reinforcement learning with Foolsball
- Reinforcement learning is learning to make decisions from experience.
- Games are a good testbed for agents to interact with an environment and explore it.
 

# About Foolsball
- 5x4 playground that provides a football/foosball-like environment.
- An agent or actor:
  - always spawned in the top-left corner
  - displayed as '⚽'
  - can move North, South, East or West.
  - can be controlled algorithmically
- A number of **static** opponents, each represented by 👕, that occupy certain locations on the field.
- A goalpost 🥅 that is fixed in the bottom right corner

## Primary goal
- We want the agent to learn to reach the goalpost 

## Secondary goals
- We may want the agent to learn to be efficient in some sense, for example, take the shortest path to the goalpost. **More precisely we want an algorithm to learn to control the agent and steer it towards the goalpost.**

In [2]:
agent = '⚽'
opponent = '👕'
goal = '🥅'

arena = [['⚽', ' ' , '👕', ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , '🥅']]

# Implementing an environment for the game of Foolsball
- OpenAI Gym has many [text environments](https://github.com/openai/gym/tree/master/gym/envs/toy_text).
- Text environments are simple to render in a notebook and super-fast to experiment with.
- We want to build out own environment for two reasons:
  - It's a great exercise in understanding the finer details, like states, actions, rewards, returns.
  - Some of the experimentation we do requires looking under the hood of the environment, which is easier with your own implementation than OpenAI Gym.
  - OpenAI Gym has a simple `step(), reset()` API that we also implement. So porting our implementation over to Gym should be easy (and fun)!



# Understanding the first bits of terminology.
## State 
- In RL, state refers to information about the environemnt and then agent.
- An RL algorithm inspects the state to decide which action to take.
- Exactly what information gets captured in `state` depends on a few factors:
  - The complexity of the environment: 
    - The number of actors, 
    - the nature of the environment, for example text or images. 
  - The complexity of the algorithm
    - A simple algorithm may only need information about the agent and its immediate surroundings.
    - A more complex algorithm may need information about the whole environment.


## Setup
- In our case we want the algorithm to only know about the location of the agent on the field. 
- We could have included information about the opponents too which would perhaps aid in the decision making but we chose not to.  

- The state therefore is a tuple: (row, col), representing the location of the agent. 
- There are 20 possible values that `state` can take on:
  - `row` can range from 0 through 4
  - `col` can range from 0 through 3

## Implementation details
- The state is actually stored as a single integer that can take on values between 0 and 19.

## Actions
The agents can perfrom actions in an environment.

## Setup
- Our agent can perform one kind of action: navigate up, down, right or left.
- It has 4 actions: 'n', 'e', 'w', 's'.

# Learning from experience
Any RL set up can be modeled as shown below:

![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTMDmrmnl_dAyjCOErHPak2gLXmQTgQnVT8gQ&usqp=CAU)

- The agent performs an action in the environment
- The state of the environment and agent change as a result
- The agent receives a reward and the updated state from the environment

## Rewards
- Reward is the signal that an agent receives after it performs an action.
- The reward structure has to be decided by us. 
- The biggest challenge of RL is that reward is often sparse. 

## Set up
- In our case the reward depends on the rules of the game and our goal.
  - If the agent runs into an opponent, the game gets over and the reward is negative (penalizes the agent).
  - If the agent makes it to the goalpost, the game gets over and the reward is positive.
  - if the agent takes the ball out of the field the reward is negative.
  - If the agent makes a valid move what shoud the reward be?

## Implementation
- The default reward structure in our case is  `{'unmarked':-1, 'opponent':-5, 'outside':-1, 'goal':+5}`
- This can be changed at any time by calling `set_rewards()`.
- Taking the ball to an unmarked position seems like a small step towards reaching the goalpost. Why would we then ever want to have a negative reward for this type of manouver?

# Let's start!
---

# Step 1: Build the Foolsball environment
The code below provides a skeleton for the **Foolsball** environment we want our agent to train in. Fill in the code marked with #Todo to create a working environment.

1. Go to the `__init__()` method and try to understand what it is doing
  1. Look at the `deserialize()` method and complete all todos.
      1. Complete the `__to_state_()` and `__to_indices__()` methods.
2. Complete the `reset()` method.
3. Go to the `step()` method and understand its intended behavior.
  1. Complete `__get_next_state_on_action__()`
  2. Complete `__get_reward_for_transition__()`
  3. Complete the `step()` method.


4. Read through the `render()` method to understand how we display the environment in the different situations. 

5. Execute the cell below and make sure there are no errors.

In [3]:
import numpy as np

class Foolsball(object):

  def __to_state__(self,row,col):
    """Convert from integer state to indices (row,col)."""
    return row*self.n_cols + col

  def __to_indices__(self, state):
    """Convert indices(row,col) to state (single integer)."""
    row = state // self.n_cols
    col = state % self.n_cols
    return row,col

  def __deserialize__(self,map:list,agent:str,opponent:str, goal:str):
    """Convrt a string representation of a map into a 2D numpy array
    Param map: list of lists of strings representing the player, opponents and goal.
    Param agent: string representing the agent on the map 
    Param opponent: string representing every instance of an opponent player
    Param goal: string representing the location of the goal on the map
    """
    ## Capture dimensions and map.
    self.n_rows = len(map)
    self.n_cols = len(map[0])
    self.n_states = self.n_rows * self.n_cols
    self.map = np.asarray(map)

    ## Store string representations for printing the map, etc.
    self.agent_repr = agent
    self.opponent_repr  = opponent
    self.goal_repr = goal

    ## Find initial state, the desired goal state and the state of the opponents. 
    self.init_state = None
    self.goal_state = None
    self.opponents_states = []

    for row in range(self.n_rows):
      for col in range(self.n_cols):

        if map[row][col] == agent:
          # Store the initial state outside the map.
          # This helps in quickly resetting the game to the initial state and
          # also simplifies printing the map independent of the agent's state. 
          self.init_state = self.__to_state__(row,col)
          self.map[row,col] = ' ' 
        
        elif map[row][col] == opponent:
          self.opponents_states.append(self.__to_state__(row,col))

        elif map[row][col] == goal:
          self.goal_state = self.__to_state__(row,col)

    assert self.init_state is not None, print(f"Map {map} does not specify an agent {agent} location")
    assert self.goal_state is not None,  print(f"Map {map} does not specify a goal {goal} location")
    assert self.opponents_states,  print(f"Map {map} does not specify any opponents {opponent} location")

    return self.init_state


  def __get_next_state_on_action__(self,state,action):
    """Return next state based on current state and action."""
    row, col = self.__to_indices__(state)
    action_to_index_delta = {'n':[-1,0], 'e':[0,+1], 'w':[0,-1], 's':[+1,0]}

    row_delta, col_delta = action_to_index_delta[action]
    new_row , new_col = row+row_delta, col+col_delta

    ## Return current state if next state is invalid
    if not(0<=new_row<self.n_rows) or not(0<=new_col<self.n_cols):
      return state  

    ## Construct state from new row and col and return it.    
    return self.__to_state__(new_row, new_col)


  def __get_reward_for_transition__(self,state,next_state):
    """ Return the reward based on the transition from current state to next state. """
    ## Transition rejected due to illegal action (move)
    if next_state == state:
      reward = self.rewards['outside']
    
    ## Goal!
    elif next_state == self.goal_state:
      reward = self.rewards['goal']
    
    ## Ran into opponent. 
    elif next_state in self.opponents_states:
      reward = self.rewards['opponent']

    ## Made a safe and valid move.   
    else:
      reward = self.rewards['unmarked']

    return reward


  def __is_terminal_state__(self, state):
    return (state == self.goal_state) or (state in self.opponents_states) 

  
  def __init__(self,map,agent,opponent,goal):
    """Spawn the world, create variables to track state and actions."""
    # We just need to track the location of the agent (the ball)
    # Everything else is static and so a potential algorithm doesn't 
    # have to look at it. The variable `done` flags terminal states.
    self.state = self.__deserialize__(map,agent,opponent,goal)
    self.done = False
    self.actions = ['n','e','w','s']

    # Set up the rewards
    self.default_rewards = {'unmarked':-1, 'opponent':-5, 'outside':-1, 'goal':+5}
    self.set_rewards(self.default_rewards)



  def reset(self):
    """Reset the environment to its initial state."""
    # There's really just two things we need to reset: the state, which should
    # be reset to the initial state, and the `done` flag which should be 
    # cleared to signal that we are not in a terminal state anymore, even if we 
    # were earlier. 
    self.state = self.init_state
    self.done  = False
    return self.state

  
  def set_rewards(self,rewards):
    if not self.state == self.init_state:
      print('Warning: Setting reward while not in initial state! You may want to call reset() first.')
    for key in self.default_rewards:
      assert key in rewards, print(f'Key {key} missing from reward.') 
    self.rewards = rewards

  
  def step(self,action):
    """Simulate state transition based on current state and action received."""
    assert not self.done, \
    print(f'You cannot call step() in a terminal state({self.state}). Check the "done" flag before calling step() to avoid this.')
    next_state = self.__get_next_state_on_action__(self.state, action)

    reward = self.__get_reward_for_transition__(self.state, next_state)

    done = self.__is_terminal_state__(next_state)

    self.state, self.done = next_state, done
    
    return next_state, reward, done


  def render(self):
    """Pretty-print the environment and agent."""
    ## Create a copy of the map and change data type to accomodate
    ## 3-character strings
    _map = np.array(self.map, dtype='<U3')

    ## Mark unoccupied positions with special symbol.
    ## And add extra spacing to align all columns.
    for row in range(_map.shape[0]):
      for col in range(_map.shape[1]):
        if _map[row,col] == ' ':
          _map[row,col] = ' + '
        
        elif _map[row,col] == self.opponent_repr: 
          _map[row,col] =  self.opponent_repr + ' '
        
        elif _map[row,col] == self.goal_repr:
          _map[row,col] = ' ' + self.goal_repr + ' '
      
    ## If current state overlaps with the goal state or one of the opponents'
    ## states, susbstitute a distinct marker.
    if self.state == self.goal_state:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' 🏁 '
    elif self.state in self.opponents_states:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' ❗ '
    else:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' ' + self.agent_repr
    
    for row in range(_map.shape[0]):
      for col in range(_map.shape[1]):
        print(f' {_map[row,col]} ',end="")
      print('\n') 
    
    print()



# Step 2: Verify the environment
Execute the two cell below and ensure that there are no runtime error and the rendering happens correctly. You should see output like this:

```
  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  

```

In [4]:
foolsball = Foolsball(arena, agent, opponent, goal)

In [5]:
foolsball.render()

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  




# Step 3: Explore the environment
- Run the next cell to play with the environment and score a few goals. 
- If there are any errors you may want to go back and update the code for the `Foolsball` class. 
- Make sure to run the cell containing `foolsball = Foolsball(arena, agent, opponent, goal)` if you update the class.

In [ ]:
## Move: n,s,e,w
## Reset: r
## Exit: x
while True:
  try:
    act = input('>>')

    if act in foolsball.actions:
      print(foolsball.step(act))
      print()
      foolsball.render()
    elif act == 'r':
      print(foolsball.reset())
      print()
      foolsball.render()
    elif act == 'x':
      break
    else:
      print(f'Invalid input:{act}')
  except Exception as e:
    print(e)

# Step 4: Understand the concept of returns
- Complete the `get_return()` function.
- Calculate returns for a few sample paths by running the next few cells

In [6]:
## Reward and return
path1 = ['e','s','e','s','s','s','e']
path2 = ['s','e','e','s','s','s','e']
path3 = ['s','s','s','e','e','s','e']
path4 = ['s','s','s','s','n','e','e','s','e']

In [ ]:
def get_return(path):
  foolsball.reset()
  foolsball.render()
  
  _return_ = 0
  for act in path: 
    next_state, reward, done = foolsball.step(act)
    foolsball.render()
    _return_ += reward
    
    if done:
      break
    
  print(f'Return (accumulated reward): {_return_}')

In [ ]:
get_return(path1)

In [ ]:
get_return(path2)

In [ ]:
get_return(path3)

In [ ]:
get_return(path4)

# Step 5: Experiment with a different reward structure.
- Does it encourage the agent to take the shortest route?

In [7]:
## Different reward structure
foolsball.set_rewards({'unmarked':0, 'opponent':-5, 'outside':-1, 'goal':+5})

In [ ]:
get_return(path1)

In [ ]:
get_return(path4)

# Step 6: Learn about discounted returns
- Get introduced to discounted return as a means to set acceptable time horizons.
$$Discounted\ Return = R_{t_1} + \gamma*R_{t_2} + \gamma^2*R_{t_3} + ... + \gamma^{n-1}*R_{t_n}$$
where $R_{t_k}$  is the reward after step `k` and $\gamma$ is called the discount factor. 
- Complete the code below to implement discounted returns.
- The discount factor $\gamma$ is a hyperparameter (why?) often set to 0.9 
😜
- Run the next few cells to see if discounting indeed has the effect we want (shorter paths have higher rewards)

In [8]:
def get_discounted_return(path, gamma=0):
  foolsball.reset()
  foolsball.render()
  _return_ = 0
  discount_coeff = 1
  for act in path: 
    next_state, reward, done = foolsball.step(act)
    _return_ += discount_coeff*reward
    discount_coeff *= gamma    
    
    foolsball.render()
    if done:
      break
    
  print(f'Return (accumulated reward): {_return_}')

In [9]:
HYPER_PARAMS = {'gamma':0.9}

In [10]:
get_discounted_return(path1, HYPER_PARAMS['gamma'])

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    ⚽  👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    ⚽   +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    ⚽  👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    ⚽   +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): 2.6572050000000007


In [ ]:
get_discounted_return(path4, HYPER_PARAMS['gamma'])

# Step 7: Formalizing the problem:
- We want the agent to reach the goalpost AND attain the highest **discounted return**.
- This means making safe and efficient moves
  - Running into opponents means game over
  - Repeated 'outsides' means inefficiency
  - Long detours are also inefficient

## The Conundrum
- We already know how to compute the discounted return for a single path.
- We can generate all possible paths and calculate their returns and pick a path with the highest return.

- Alas there are too many paths (4 possible decisions at each step => combinatorial explosion).


## The "Trick"
- Even though there are too many paths, all of them are made up of a smaller number of (state,action) pairs.
- We can calculate the return for each of the 80(=20x4) state action pairs.
- To emphasize, we want to calculate the return for each (state,action) pair, not just the reward.
  - Calculating return means peeking into the future, beyond that (state,action) pair.


## Todo:
- As a precursor to calculating returns for every (state,action) pair, let's try to calculate the reward for every (state,action) pair.

- Understand how the code in the next two cells creates a Pandas table to store the rewards for every (state, action) pair.

- We will cheat a little by using private methods of the `Foolsball` class
  - Use the `__get_next_state_on_action__()` and `__get_reward_for_transition__()` methods to complete the code in the third cell below.
  - Run the fourth cell to view the rewards table. 
  - Notice that rewards for terminal states are kept undefined since no actions are allowed in those states.



In [11]:
import pandas as pd

In [12]:
REWARDS_TBL = pd.DataFrame.from_dict({s:{a:None for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
REWARDS_TBL

,n,e,w,s
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,None,None,None,None
8,None,None,None,None
9,None,None,None,None


In [13]:
for state in REWARDS_TBL.index:
  if not foolsball.__is_terminal_state__(state): #Only calculate rewards for non-terminal states
    for action in REWARDS_TBL.columns:
      next_state = foolsball.__get_next_state_on_action__(state,action)
      REWARDS_TBL.loc[state, action] = foolsball.__get_reward_for_transition__(state, next_state)

In [14]:
terminal_states = foolsball.opponents_states+[foolsball.goal_state]
print(terminal_states)
REWARDS_TBL

[2, 7, 9, 15, 17, 19]


,n,e,w,s
0,-1,0,-1,0
1,-1,-5,0,0
2,None,None,None,None
3,-1,-1,-5,-5
4,0,0,-1,0
5,0,0,0,-5
6,-5,-5,0,0
7,None,None,None,None
8,0,-5,-1,0
9,None,None,None,None


#Step 8: Learn about returns table
Create a returns table (no TODOs here)
- Run the next four cells and guess why we are setting the returns for terminal states to 0.
  - We leave the returns for all non-terminal states undefined.
  - Trying to fill up these entries will be the focus of the rest of the notebook.

- A function to create new instances of the returns table is also provided in the fourth cell below.

In [15]:
RETURNS_TBL = pd.DataFrame.from_dict({s:{a:None for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

In [16]:
RETURNS_TBL

,n,e,w,s
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,None,None,None,None
8,None,None,None,None
9,None,None,None,None


In [17]:
RETURNS_TBL.loc[terminal_states]

,n,e,w,s
2,None,None,None,None
7,None,None,None,None
9,None,None,None,None
15,None,None,None,None
17,None,None,None,None
19,None,None,None,None


In [18]:
RETURNS_TBL.loc[terminal_states] = 0
RETURNS_TBL

,n,e,w,s
0,None,None,None,None
1,None,None,None,None
2,0,0,0,0
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


In [19]:
def make_returns_table(terminal_states):
  """Create an empty returns table."""
  table = pd.DataFrame.from_dict({s:{a:None for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
  table.loc[terminal_states] = 0
  return table

# Step 9: Use dynamic programming to fill up the returns table.
- The return for a (state, action) can be defined in terms of returns of the next state. 
  - $Return(state_t,action_t) = Reward(state_t,state_{t+1}) + \gamma * \max[Return(state_{t+1}, action=='n'),\\ Return(state_{t+1}, action=='e'), \\ Return(state_{t+1}, action=='w'), \\ Return(state_{t+1}, action=='s')]$
  - Can you see why this ought to be?

  - This motivates the use of dynamic programming to fill up the returns table.  

## Todo:
- Read the code in the next cell and try to understand the dynamic programming based solution. 
- Run the code in the next cell. The code causes a stack overflow. Why?
- Pass debug= True to see what the problem is.

In [20]:
def fill_returns_table_v0(table,state,debug=False): 
  """ Recursively fill a returns table, one state at a time."""
  for action in table.columns:
    if table.loc[state][action] is None:
      next_state = foolsball.__get_next_state_on_action__(state, action)
      reward = foolsball.__get_reward_for_transition__(state, next_state)

      if debug:
        print(f'Trying to fill ({state},{action},{next_state})')
      
      fill_returns_table_v0(table, next_state, debug) # <= Earth shaking problem here!!! 😱😱😱
      table.loc[state][action]  = reward + HYPER_PARAMS['gamma'] * table.loc[next_state].max()
    
    else:
      if debug:
        print((state,action),f'already has a RETURN {table.loc[state][action]}')
  

In [21]:
table = make_returns_table(terminal_states)
fill_returns_table_v0(table,state=0, debug=True)

Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to fill (0,n,0)
Trying to f

RecursionError: ignored

## Contd..
- The code above crashed becasue of indefinite recursion caused by a state,action pairs that resulted in the next state being the same as the current state
- We can fix this by catching this case and a returning a large negative return.
- Why is the large negative return necessary?
- Look at the code below that tries to mitigate this problem with the following code:
```
      if next_state == state:
        table.loc[state][action] = -np.inf # <= No self recursion
```

- Run the next two cells to see if how the RETURNS table gets filled up.

In [22]:
def fill_returns_table_v1(table,state,debug=False):
  for action in table.columns:
    if table.loc[state][action] is None:
      next_state = foolsball.__get_next_state_on_action__(state, action)
      reward = foolsball.__get_reward_for_transition__(state, next_state)
      
      if debug:
        print(f'Trying to fill ({state},{action},{next_state})')
      
      if next_state == state:
        table.loc[state][action] = -np.inf # <= No self recursion
      else:
        fill_returns_table_v1(table,next_state,debug)
        table.loc[state][action]  = reward + HYPER_PARAMS['gamma'] * table.loc[next_state].max()
    else:
      if debug:
        print((state,action),f'already has a RETURN {table.loc[state][action]}')

In [23]:
table = make_returns_table(terminal_states)
fill_returns_table_v1(table, state=0, debug=False)

RecursionError: ignored

## Contd..
- The code above crashed becasue of indefinite mutual recursion caused by a (state,action) pair that resulted in the (next_state, next_action) bringing us back to the first state.
- We can fix this by deferring these cases.
- Let' see if we can get somewhere.
- Run the next few cells to find out.

In [24]:
def fill_returns_table_v2(table,state, debug=False):
  for action in table.columns:
    if table.loc[state][action] is None:
      next_state = foolsball.__get_next_state_on_action__(state, action)
      reward = foolsball.__get_reward_for_transition__(state, next_state)
      
      if debug:
        print(f'Trying to fill ({state},{action},{next_state})')
      
      if next_state == state:
        table.loc[state][action] = -np.inf # <= No self recursion
      
      elif not table.loc[next_state].isna().any(): # <= No recursion beyond immediate neighbor!
        fill_returns_table_v2(table, next_state)
        table.loc[state][action]  = reward + HYPER_PARAMS['gamma'] * table.loc[next_state].max()
    
    else:
      if debug:
        print((state,action),f'already has a RETURN {table.loc[state][action]}')

In [25]:
table = make_returns_table(terminal_states)

In [26]:
fill_returns_table_v2(table,state=0)

In [27]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,None,None,None,None
2,0,0,0,0
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


## Contd...
- We were able to fill up `(0,'n')` and `(0,'w')`.
- Let's try to fill up some of the cells for state `1`.

In [28]:
fill_returns_table_v2(table,state=1)

In [29]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


## Contd...
- Non-recursive state,action pairs are fille up for states `0` and `1`.
- We will skip over state `2` as it is a terminal state.
- Let's try to fill up the table for state `3`.

In [30]:
fill_returns_table_v2(table,state=3)

In [31]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


## Contd...
- Let's try to fill in the rest of the states too. 

In [32]:
for s in range(4,19):
  fill_returns_table_v2(table,state=s)

In [33]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,-inf,None
5,None,None,None,-5
6,-5,-5,None,None
7,0,0,0,0
8,None,-5,-inf,None
9,0,0,0,0


In [34]:
# Count number of Nones
table.isna().sum()

n    8
e    6
w    6
s    8
dtype: int64

## Contd...
- Let's make another pass over all the states to see if some of the cells deferred earlier can be filled now.

In [35]:
for s in range(0,19):
  fill_returns_table_v2(table,state=s)

In [36]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,-inf,None
5,None,None,None,-5
6,-5,-5,None,None
7,0,0,0,0
8,None,-5,-inf,None
9,0,0,0,0


In [37]:
table.isna().sum()

n    8
e    6
w    6
s    8
dtype: int64

## Contd...
- The second pass does not seem to have made any updates.
- Let's confirm this by making a third pass over all the states.

In [38]:
for s in range(0,19):
  fill_returns_table_v2(table,state=s)

In [39]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,-inf,None
5,None,None,None,-5
6,-5,-5,None,None
7,0,0,0,0
8,None,-5,-inf,None
9,0,0,0,0


In [40]:
table.isna().sum()

n    8
e    6
w    6
s    8
dtype: int64

## Contd...
- Unfortunately, it seems that cells will mutual recursion like `(0,'e')` <=> `(1,'w')` never get filled by this algorithm. 
- The rest of the notebook will try to come up with a non-mutually-recursive solution to the problem of filling up the RETURNS table.

# Get Some more Coffee
---

# Step 10: Estimating returns through simulation
- and Monte Carlo sampling
- No more dynamic programming 
- No more cheating by peeping into the environment (private APIs)
- Sampling requires simulating the game and trying random actions.

## Todo
- Run the code in the next two cells to collect and print a random episodes.
  - The episode starts with the environment in the initial state 
  - The agent tries random actions
  - The episode terminates when the agent collides with an opponent or reaches the goalpost.


In [41]:
def collect_random_episode():
  state = foolsball.reset()
  done = False
  episode = []

  while not done:
    action = np.random.choice(foolsball.actions)
    next_state, reward, done = foolsball.step(action)
    episode.append([state, action, reward])
    state = next_state
  
  return episode

In [42]:
ep = collect_random_episode()
foolsball.render()
print(ep)

  +    +    ❗    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


[[0, 's', 0], [4, 'w', -1], [4, 'e', 0], [5, 'n', 0], [1, 'n', -1], [1, 's', 0], [5, 'n', 0], [1, 'e', -5]]


# Step 11: Implement discounted returns for episodes
- Complete the function `discounted_return_from_episode()` that computes the discounted return for every (state,action) pair in an episode.
  - If an episode is:  $(s_1,a_1,r_1), (s_2,a_2,r_2), (s_3, a_3, r_3)$, **excluding the terminal state**:
  - The (discounted) return for $s_1$ is $r_1 + \gamma * r_2 + \gamma^2 * r_3$
  - The (discounted) return for $s_2$ is $r_2 + \gamma * r_3$
  - The (discounted) return for $s_3$ is $r_3$ 

- Run the next couple of cells to print discounted returns for entire episodes.


In [43]:
def discounted_return_from_episode(ep, gamma=0):
  states, actions, rewards = list(zip(*ep))
  rewards = np.asarray(rewards)
  discount_coeffs = np.asarray([np.power(gamma,p) for p in range(len(rewards))])
  
  l = len(rewards)
  discounted_returns = [np.dot(rewards[i:],discount_coeffs[:l-i]) for i in range(l)]

  return (states, actions, discounted_returns)


In [44]:
discounted_return_from_episode(ep, gamma=HYPER_PARAMS['gamma'])

((0, 4, 4, 5, 1, 1, 5, 1),
 ('s', 'w', 'e', 'n', 'n', 's', 'n', 'e'),
 [-3.9475845000000005,
  -4.386205,
  -3.7624500000000003,
  -4.1805,
  -4.6450000000000005,
  -4.050000000000001,
  -4.5,
  -5.0])

# Step 12: Estimate returns by simulating lots of episodes.
- The code below creates two tables:
  - ESTIMATED_RETURNS_TBL for accumulating the return for every (state,action) pair across all episodes.
  - VISITS_COUNTS_TBL for storing the number of times a (state,action) pair appears across all episodes.

- It then runs an algorithm to generate episodes and fill in these tables.

Here's the idea:
- Create many random episodes
  - Examine each (state, action) pair in every episode.
  - Calculate and accumulate the return for this pair
    - Since we have the full episode, we can "see the future" and calculate the return.
    - The return for a (state,action) pair is just a (very bad) estimate of the "real" return, since we are looking at just one of the many paths that could possible contain the (state,action).
  - Record the visit count of the (state, action) pair.   

- At the end the we divide the accumulated returns by the visit counts to get an averaged estimate of the retruns. 


## Todo:
- Complete the code in the **next two cells** to implement what's known as Monte Carlo estimation.
- Run the cells to see how well the alorithm fares.
- Does the algorithm generate sensible looking returns (estimates)?

In [45]:
# Create empty returns table 
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 100  #Try 100, 500, 2000

for i in range(n_episodes):
  episode_i = collect_random_episode()
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += ret
    VISITS_COUNTS_TBL.loc[s,a] += 1
  

In [46]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
estimated_returns

,n,e,w,s
0,-4.915086,-3.950317,-4.846800,-3.630156
1,-5.030212,-4.833333,-3.832372,-3.592341
2,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000
4,-3.960664,-3.751971,-4.516873,-3.809151
5,-3.716289,-3.942166,-3.503796,-4.750000
6,-4.545455,-4.285714,-3.606060,-3.288939
7,0.000000,0.000000,0.000000,0.000000
8,-3.444817,-4.782609,-4.602253,-3.500443
9,0.000000,0.000000,0.000000,0.000000


# Step 13: Intro to Policies
- The estimated returns table is hard to evaluate visually.
- To use the table to make decisions, we grab the action with the highest return for each of the states in the table.
- This is called a **policy**.
- This will be a greedy policy since we take the best action at each state.
- Run the next two cells to output a greedy policy derived from the returns table filled earlier.

In [47]:
def greedy_policy_from_returns_tbl(table):
  policy = {s:None for s in table.index }

  for state in table.index:
    if state not in terminal_states:
      greedy_action_index = table.loc[state].argmax()
      greedy_action = table.columns[greedy_action_index]
      policy[state] = greedy_action

  return policy

In [48]:
policy0 = greedy_policy_from_returns_tbl(estimated_returns)

# Contd..
- Here's a function to superimpose a policy over the environment.
- Use the code in the next two cells to eyeball the policy we just generated

In [49]:
def pretty_print_policy(policy):
  direction_repr = {'n':' 🡑 ', 'e':' 🡒 ', 'w':' 🡐 ', 's':' 🡓 ', None:' ⬤ '}

  for row in range(foolsball.n_rows):
    for col in range(foolsball.n_cols):
      state = row * foolsball.n_cols + col
      print(direction_repr[policy[state]],end='')
    print()

In [50]:
pretty_print_policy(policy0)

 🡓  🡓  ⬤  🡑 
 🡒  🡐  🡓  ⬤ 
 🡑  ⬤  🡐  🡓 
 🡒  🡑  🡑  ⬤ 
 🡑  ⬤  🡒  ⬤ 


# Step 14: Exploiting the information in the returns table.
- We are improving our estimates of the returns with each successive episode. 
- But we are still generating random episodes throughout. 
- We should also exploit the information we accrue in the RETURNS table.
- The implementation below is quite similar to `collect_random_episode()` but here's the key difference:
  - In state s, `collect_random_episode()` was returning a random action from ('n', 's', 'e', 'w').
  - But from the returns table we know that one of the action, say 'e' generates the best returns so we can make a greedy choice and always return 'e'.
  - This is what `collect_greedy_episode_from_returns_tbl()` does.

- Run the next to cells to see the difference. 

In [51]:
def collect_greedy_episode_from_returns_tbl(table, max_ep_len=20):
  state = foolsball.reset()
  done = False
  episode = []

  for _ in range(max_ep_len):
    if done:
      break
    
    greedy_action_index = table.loc[state].argmax()
    greedy_action = table.columns[greedy_action_index]
    next_state, reward, done = foolsball.step(greedy_action)
    episode.append([state, greedy_action, reward])
    state = next_state
  
  return episode

In [52]:
collect_greedy_episode_from_returns_tbl(estimated_returns)

[[0, 's', 0],
 [4, 'e', 0],
 [5, 'w', 0],
 [4, 'e', 0],
 [5, 'w', 0],
 [4, 'e', 0],
 [5, 'w', 0],
 [4, 'e', 0],
 [5, 'w', 0],
 [4, 'e', 0],
 [5, 'w', 0],
 [4, 'e', 0],
 [5, 'w', 0],
 [4, 'e', 0],
 [5, 'w', 0],
 [4, 'e', 0],
 [5, 'w', 0],
 [4, 'e', 0],
 [5, 'w', 0],
 [4, 'e', 0]]

# Step 15: Using Greedy Returns
## Todo 
- Implement the loop in the cell below to update the returns table. 
- The code will be exactly what we used earlier, except that it will use greedy episodes.

- Run the next few cells to evaluate the effectiveness.


In [53]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 1000

for i in range(n_episodes):
  estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1)
  
  episode_i = collect_greedy_episode_from_returns_tbl(estimated_returns)
  #print(episode_i)
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += ret
    VISITS_COUNTS_TBL.loc[s,a] += 1

In [54]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
estimated_returns

,n,e,w,s
0,-5.759138,-3.892117,-5.759138,0.0
1,-5.607883,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
5,0.000000,0.000000,0.000000,0.0
6,0.000000,0.000000,0.000000,0.0
7,0.000000,0.000000,0.000000,0.0
8,0.000000,0.000000,0.000000,0.0
9,0.000000,0.000000,0.000000,0.0


In [55]:
policy1 = greedy_policy_from_returns_tbl(estimated_returns)

In [56]:
pretty_print_policy(policy1)

 🡓  🡒  ⬤  🡑 
 🡑  🡑  🡑  ⬤ 
 🡑  ⬤  🡑  🡑 
 🡑  🡑  🡑  ⬤ 
 🡑  ⬤  🡑  ⬤ 


# Contd... 
Using greedy returns seems to have deteriorated the returns values. Why is that? 

# Step 16: The Exploration-exploitation Dilemma

- We have tried pure exploration (with random episodes)
- We have also tried pure exploitation (with policy generated from the returns table)
- A good agent should try to balance both.


## Epsilon-greedy episodes
- An epsilon greedy episode blends the previous two approaches
- Precisely, when in state `s`:
  - The epsilon greedy episode will pick the action yielding the highest returns with a high probability, say 0.8 
  - With a low probability it can return one of the suboptimal, random actions.
  - The hyperparameter `epsilon` or $\epsilon$ decides the probability of selecting a random action and `1-epsilon` is the probability of picking the best action. 

  - Example with epsilon = 0.2 and assuming `w` is the best action
    - state `s`
    - Actions = ('n','e','w','s')
    - Best action (yielding highest return) = 'w'
    - Sampling probabilities = $[{\epsilon \over 4},{\epsilon \over 4}, 1-\epsilon+{\epsilon \over 4},{\epsilon \over 4}] = [0.05,0.05,0.85,0.05]$


## Todo:
Finish the code below and look at how the output differs from the other two methods. 


In [57]:
def collect_epsilon_greedy_episode_from_returns_tbl(table, max_ep_len=20, epsilon=0.1):
  
  state = foolsball.reset()
  done = False
  episode = []

  for _ in range(max_ep_len):
    if done:
      break
    
    actions = table.columns
    action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)
    
    greedy_action_index = table.loc[state].argmax()
    action_probs[greedy_action_index] += 1-epsilon
    
    epsilon_greedy_action = np.random.choice(table.columns,p=action_probs)

    next_state, reward, done = foolsball.step(epsilon_greedy_action)
    episode.append([state, epsilon_greedy_action, reward])
    state = next_state

  return episode

In [58]:
# Generate an epsilon-greedy episode every time. 
collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns, epsilon=1)

[[0, 's', 0],
 [4, 'n', 0],
 [0, 's', 0],
 [4, 'n', 0],
 [0, 'n', -1],
 [0, 'e', 0],
 [1, 's', 0],
 [5, 'e', 0],
 [6, 'n', -5]]

# Step 17: Epsilon-greedy updates.
## Todo:
- Run the next few cells to see the effect of using an epsilon greedy approach.

In [59]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 1000

for i in range(n_episodes):
  estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1)
  
  episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns)
  #print(episode_i)
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += ret
    VISITS_COUNTS_TBL.loc[s,a] += 1

In [60]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
estimated_returns

,n,e,w,s
0,-1.718065,-0.312800,-1.312686,-0.265109
1,-4.165814,-4.545455,-0.279372,-0.597375
2,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000
4,-0.224579,-0.419908,-1.552336,-0.502163
5,-0.529056,-0.685856,-0.362003,-4.942529
6,-4.166667,-3.750000,-0.289437,-0.883870
7,0.000000,0.000000,0.000000,0.000000
8,-0.422310,-4.722222,-2.089197,-0.348999
9,0.000000,0.000000,0.000000,0.000000


In [61]:
policy2 = greedy_policy_from_returns_tbl(estimated_returns)
policy2

{0: 's',
 1: 'w',
 2: None,
 3: 'n',
 4: 'n',
 5: 'w',
 6: 'w',
 7: None,
 8: 's',
 9: None,
 10: 'n',
 11: 'w',
 12: 'n',
 13: 'w',
 14: 's',
 15: None,
 16: 'n',
 17: None,
 18: 'n',
 19: None}

In [62]:
pretty_print_policy(policy2)

 🡓  🡐  ⬤  🡑 
 🡑  🡐  🡐  ⬤ 
 🡓  ⬤  🡑  🡐 
 🡑  🡐  🡓  ⬤ 
 🡑  ⬤  🡑  ⬤ 


# Contd...
The policy is better than the greedy policy but does not seem like a clear winner over the purely random policy. What could be going wrong?

# Step 18: Revisiting Exploration-Exploitation with Epsilon Decay

- What is the best way to balance exploitation with exploration?
  - In the beginning, pick absolutely random actions in every state.
  - Slowly reduce the randomness to a small value.

## Todo:
- In the code below pick a value of `epsilon` that makes all actions equiprobable in `collect_epsilon_greedy_episode_from_returns_tbl()`.

- Fill in the code to 'anneal' epsilon over episodes. The value of epsilon shoud not drop below the minimum threshold.

- Run the next few cells to evaluate this approach.

- Does the policy look any better?

In [63]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 10000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999

for i in range(n_episodes):
  estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1)
  
  epsilon = max(epsilon, min_epsilon)
  episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns,epsilon=epsilon)
  epsilon *= epsilon_decay
  #print(episode_i)
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += ret
    VISITS_COUNTS_TBL.loc[s,a] += 1

In [64]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
print(estimated_returns)

policy3 = greedy_policy_from_returns_tbl(estimated_returns)
print(policy3)

pretty_print_policy(policy3)

           n         e         w         s
0  -1.746405 -0.922257 -1.772489 -0.281284
1  -2.888780 -4.981685 -0.615094 -1.922937
2   0.000000  0.000000  0.000000  0.000000
3   0.000000  0.000000  0.000000  0.000000
4  -0.273176 -0.757832 -1.582938 -0.760943
5  -1.996296 -2.219021 -0.531641 -4.976190
6  -4.833333 -4.864865 -1.457448 -2.379446
7   0.000000  0.000000  0.000000  0.000000
8  -0.513681 -4.976744 -2.830149 -0.777082
9   0.000000  0.000000  0.000000  0.000000
10 -2.852932 -3.653550 -4.500000 -0.492600
11 -3.333333 -3.165833 -3.037500 -3.750000
12 -2.644928 -0.282318 -2.310787 -2.304733
13 -4.782609  1.469479 -0.876046 -4.791667
14 -1.976950 -4.583333 -2.556409  3.112912
15  0.000000  0.000000  0.000000  0.000000
16 -1.486091 -4.285714 -3.218547 -2.028391
17  0.000000  0.000000  0.000000  0.000000
18  2.250000  4.926471 -4.285714  2.800000
19  0.000000  0.000000  0.000000  0.000000
{0: 's', 1: 'w', 2: None, 3: 'n', 4: 'n', 5: 'w', 6: 'w', 7: None, 8: 'n', 9: None, 10: 's', 11: 

# Step 19: Constant Alpha

## The idea:
- Dividing the accumulated returns by visit count has a non linear effect on the updates. (Go back to previous step and see for yourself).

- Don't divide at all!

- But we need to ensure that updates are small

- Idea: 

  - `ESTIMATED_RETURNS_TBL.loc[s,a]` and `ret` are both estimates of the same quantity. 

  - Use the difference of the two estimates to update `ESTIMATED_RETURNS_TBL.loc[s,a]` much like we do in Deep Learning.


## Todo:
- Complete the missing code in the next cell.
- Run the next few cells to get a policy and evaluate it.
- Does the policy help the agent attain its goal?


In [65]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 10000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999

alpha = 0.01

for i in range(n_episodes):
  estimated_returns = ESTIMATED_RETURNS_TBL
  
  epsilon = max(epsilon,min_epsilon)
  episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns,epsilon=epsilon)
  epsilon *= epsilon_decay
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += alpha*(ret - ESTIMATED_RETURNS_TBL.loc[s,a])

In [66]:
estimated_returns = ESTIMATED_RETURNS_TBL
print(estimated_returns)

policy4 = greedy_policy_from_returns_tbl(estimated_returns)
print(policy4)

pretty_print_policy(policy4)

           n         e         w         s
0   0.313897  1.221862  0.182514  1.477894
1  -2.086330 -4.514337  1.504675 -1.585462
2   0.000000  0.000000  0.000000  0.000000
3   0.000000  0.000000  0.000000  0.000000
4   1.233655  1.071870  0.369676  1.700201
5  -1.412414 -1.247453  1.394268 -4.406212
6  -1.264140 -1.264140 -1.083396 -0.113837
7   0.000000  0.000000  0.000000  0.000000
8   1.331715 -4.873686  0.557415  2.158373
9   0.000000  0.000000  0.000000  0.000000
10 -0.201627 -0.351116 -0.827431  3.017597
11 -0.245050 -0.413174  0.072333 -0.245050
12  1.564360  2.531290  0.901244  1.221199
13 -4.457565  3.114203  2.005124 -4.295578
14  2.510405 -4.375611  2.611234  4.112697
15  0.000000  0.000000  0.000000  0.000000
16  1.671439 -1.149784 -0.780497 -0.614327
17  0.000000  0.000000  0.000000  0.000000
18  3.175740  5.000000 -4.164330  2.762422
19  0.000000  0.000000  0.000000  0.000000
{0: 's', 1: 'w', 2: None, 3: 'n', 4: 's', 5: 'w', 6: 's', 7: None, 8: 's', 9: None, 10: 's', 11: 

#Step 20: Dabble in OpenAI Gym

In [67]:
import gym

In [69]:
taxi_env = gym.make('Taxi-v3')

In [70]:
taxi_env.reset()

329

Look at the [implementation](https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py) to figure out how the environment encodes the 500 states (25 for car, 5 for passenger and 4 for destination) into a single integer

In [71]:
taxi_env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In the rendering above, yellow is the taxi, the pickup is highlighted in Blue whereas the dropoff is in Magenta. The four possible pickup/drop off points are represented as follows before being encoded as part of the state:

    Passenger locations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
    - 4: in taxi
    
    Destinations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)

Take a random action. Sampling is built into Gym action spaces.

In [72]:
taxi_env.step(taxi_env.action_space.sample())

(349, -1, False, {'prob': 1.0})

Let's also try to decode the state to a more interpretable form: a 4-tuple

In [82]:
next_state, reward, done, _ = taxi_env.step(taxi_env.action_space.sample())
taxi_env.render()
print(list(taxi_env.decode(next_state)), reward, done)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
[2, 3, 2, 1] -1 False


Number of actions and states.

In [83]:
print(taxi_env.observation_space.n, taxi_env.action_space.n)

500 6


Redefine the `collect_epsilon_greedy_episode_from_returns_tbl()` to use `taxi_env`. Ideally we should instead pass this as a paramter.

In [84]:
def collect_epsilon_greedy_episode_from_returns_tbl(table, max_ep_len=20, epsilon=0.1):
  
  state = taxi_env.reset()
  done = False
  episode = []

  for _ in range(max_ep_len):
    if done:
      break
    
    actions = table.columns
    action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)
    
    greedy_action_index = table.loc[state].argmax()
    action_probs[greedy_action_index] += 1-epsilon
    
    epsilon_greedy_action = np.random.choice(table.columns,p=action_probs)
    

    next_state, reward, done, _ = taxi_env.step(epsilon_greedy_action)
    episode.append([state, epsilon_greedy_action, reward])
    state = next_state

  return episode

Use the constant alpha version that we used to solve Foolsball. To keep track of progress we print the running average return of 100 successive episodes. The code below will take a long time (~10 min). You can also try with a shorter `max_ep_len`. 

In [85]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in range(taxi_env.action_space.n)} for s in range(taxi_env.observation_space.n)}, orient='index')

n_episodes = 10000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999

alpha = 0.01
avg_episode_returns = 0

for i in range(n_episodes):
  estimated_returns = ESTIMATED_RETURNS_TBL
  
  epsilon = max(epsilon,min_epsilon)
  episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns,epsilon=epsilon,max_ep_len=100)
  #print(episode_i)
  epsilon *= epsilon_decay
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])
  avg_episode_returns += discounted_returns[0]
  if (i+1) % 100 == 0:
    print(avg_episode_returns/100)
    avg_episode_returns = 0

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += alpha*(ret - ESTIMATED_RETURNS_TBL.loc[s,a])

-37.916244291731665
-38.80922096556432
-37.77897088716877
-35.222728386497415
-32.561286645459035
-31.74652720741585
-29.73059454101195
-27.8094877818274
-28.493962560794674
-27.33160270070855
-23.894191616431534
-25.04281421168457
-19.545593984065317
-18.859464208371122
-21.22459595809106
-16.2778472427007
-12.749867428086816
-13.336566128528139
-12.551000032676777
-9.78654190619256
-9.779184103031792
-9.996676947930304
-10.674446999835013
-8.919623566823072
-9.031596916983858
-10.108688146410689
-9.015339721823883
-9.593258344068186
-8.567749572142393
-8.850193496215551
-9.521867547253668
-9.273027703071062
-8.352698552780057
-8.372572222012238
-9.288408335381305
-8.50125313470903
-8.659233502356775
-7.976743179136402
-7.896162505724316
-8.114015523190817
-8.068943048187906
-9.209120450270394
-8.315680389850497
-8.706772625004026
-7.653266448482506
-9.129745782099137
-9.021108086150724
-7.455864889374184
-9.445137365883534
-7.885217224196865
-8.857692601471426
-8.450637299864688
-7.6

 The return is still negative. Have we made progress?

In [86]:
ESTIMATED_RETURNS_TBL

,0,1,2,3,4,5
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-11.202997,-12.556741,-11.550832,-9.367088,-6.673317,-15.921330
2,-5.486084,-5.976285,-5.502165,-4.049049,5.424747,-6.802176
3,-10.537519,-9.816602,-9.113138,-9.625904,-0.178755,-10.576995
4,-11.978471,-12.128598,-12.237652,-12.395372,-19.974012,-22.183048
...,...,...,...,...,...,...
495,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
496,-11.283901,-9.819963,-9.251280,-9.339794,-27.288536,-13.415172
497,-4.482728,-3.763162,-4.346565,-2.637178,-4.780068,-7.501932
498,-7.035032,-6.854581,-8.136320,-6.422699,-7.220164,-13.929852


Lets see how good the policy is. Run the code below a few times to see how often the policy gets the passenger to the dropoff.

In [88]:
state = taxi_env.reset()
done = False

while not done:
  action = ESTIMATED_RETURNS_TBL.columns[ESTIMATED_RETURNS_TBL.loc[state].argmax()]
  next_state, reward, done, _ = taxi_env.step(action)
  taxi_env.render()
  print(list(taxi_env.decode(state)),action,reward)
  state = next_state

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
[1, 0, 2, 0] 0 -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
[2, 0, 2, 0] 0 -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
[3, 0, 2, 0] 0 -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
[4, 0, 2, 0] 4 -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (North)
[4, 0, 4, 0] 1 -1
+---------+
|R: | : :G|
| : | : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
[3, 0, 4, 0] 1 -1
+---------+
|R: | : :G|
|_: | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
[2, 0, 4, 0] 1 -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
[1, 0, 4, 0] 1 -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
[0, 0, 

# Step 21: What to do next?
- Make the world stochastic.
  - When in state `s`, action `a` might result in more than one outcomes.
  - For example `(0,'e')` might take the agent to state `1` with probability 0.9 and, with probability 0.066, (try to )take the agent north or west, brining it back to state '0', or with proobability 0.033 take the agent south to state `4`. 


- Make the world dynamic.
  - The agent can spawn in any of the unoccupied cells.
  - The opponents can move, perhaps chase our agent.
    - What extra information might we need to design a good algorithm. 

- Don't shift the goalpost though. 😛

- The OpenAI Gym environment defines this [criterion](https://gym.openai.com/envs/Taxi-v1/) to consider the environment solved.
  - Implement this definition to gauge the progress of your algorithm. 

- Update the REWARDS table and use the updated table after each step, not after each episode. 
  - This is known as [SARSA](https://lilianweng.github.io/lil-log/2018/02/19/a-long-peek-into-reinforcement-learning.html#sarsa-on-policy-td-control) because it uses (S)tate, (A)ction, (R)eward, Next (S)state and Next (A)ction at every step to update the RETURNS table.


# Notes:
- The Returns table is known as a $Q$ Table in RL literature. 
- The values in the returns table are called $action\ values: A$.
- The action values corresponding to the highest return are called the $state\ values: V$ 
- The policy at any step is often denoted by $\pi$
- The optimal values of $A$, $V$ and $\pi$ are denoted by $A^*$, $V^*$ and $\pi^*$
- Reinforcement learning algorithms usually start with a randomly initialized $Q$ table/action values $A$, then use it to calculate $V$ and then generate a policy $\pi$ from $V$. The policy is then used to sample many episodes and get better estimates for $Q$. This can be repeated many many times until we get a good-enough policy.
- Read the section titled: **Neural Networks and Deep Reinforcement Learning** [here](https://wiki.pathmind.com/deep-reinforcement-learning) to understand how deep learning fits into the picture.
- Check out this [PDF](https://ieor8100.github.io/rl/docs/RL%20in%20Robotics.pdf) to see how robotics is benefitting from DRL.


# References
- https://lilianweng.github.io/lil-log/2018/02/19/a-long-peek-into-reinforcement-learning.html
- https://spinningup.openai.com/en/latest/
- https://gym.openai.com/
